# 抓取0daytown的百度链接
## 目的
得到大型软件、工具软件教程的百度盘链接，以后方便学习
申请1个月的会员，然后用BeautifulSoup进行抓取，存入xls文件

## 要点
* 分3步
    抓取所有cat号，找出名称catlist
    抓取/cat=xx界面下的所有网页 pagelist
    抓取/p=xx界面下的链接地址 

* 保存为csv格式，
    类型
    文件名
    文件网页
    百度盘地址
    百度盘密码

* 几个方法：
    * xpath找到需要的地址
    * find("a")[href]
    
* 还有一个方法就是直接对http://www.0daytown.com/?p=XXXX 进行解析，关于类别就是抓取这个页面里的
    找到目前最大的http://www.0daytown.com/?p=107109, (191219)，对这个网页进行解析即可

* 要解决抓取过程中没有链接的情况

* 有防爬虫机制，必须更换cookies

* df.append(data)是错误的，而应该进行赋值 df = df.append(data)。

* #将List转为字符串
        baidulinkstr=';'.join(baidulinklist) 
        
* 下载过程中失去链接的处理方法
    from requests.adapters import HTTPAdapter
    s = requests.Session()
    s.mount('http://', HTTPAdapter(max_retries=3))
    r = s.get('http://www.google.com.hk', timeout=5)
    
* glob模块是最简单的模块之一，内容非常少。用它可以查找符合特定规则的文件路径名。跟使用windows下的文件搜索差不多。查找文件只用到三个匹配符：”*”, “?”, “[]”。”*”匹配0个或多个字符；”?”匹配单个字符；”[]”匹配指定范围内的字符，如：[0-9]匹配数字。

* ReadTimeoutError: 出错后，如何重新连接？
    ConnectionError: HTTPConnectionPool(host='www.0daytown.com', port=80): Read timed out.


* '''
测试HTTP time out 怎么办，如何重新链接
ReadTimeoutError: HTTPConnectionPool(host='www.0daytown.com', port=80): Read timed out.

During handling of the above exception, another exception occurred:

ConnectionError                           Traceback (most recent call last)

ConnectionError: HTTPConnectionPool(host='www.0daytown.com', port=80): Read timed out.
'''
'''
解决方法：
1.使用while循环，遇到网络中断修改pagenumber, 。这里用for循环不可以，因为无法修改循环计数
2.用一级except即可，不要用。注意要将所有的except都包括，如果有遗漏将会报错
'''


In [1]:

import requests
import unittest
from bs4 import BeautifulSoup

import urllib3
from selenium import webdriver
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

import socket

import pandas as pd

import lxml

import urllib
import random   #随机生成休眠时间
import time

from requests.adapters import HTTPAdapter

import datetime

In [2]:


User_Agent= [
"Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
        "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
        "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
        "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0"
    ]                                #User_Agent的集合


headers1={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:55.0) Gecko/20100101 Firefox/55.0",
         'Cookie': '__cfduid=d71bfdd08a3066b3a91e22850afd06bff1576705865; PHPSESSID=m42hfkvmhr26d2j6rp9aqs81l6; wordpress_test_cookie=WP+Cookie+check; wordpress_logged_in_d1bdccf8f22a41290379013c104952b6=aaronyinyong%7C1577971950%7Co5ISvGpK8eYBZVUk9PuKzXcgasandHEyNu93pybQ9FK%7C457fb1309b2853366c70b4ad57b418e14fd25873dd2fbbb91a3daa45928f38e2'}
# send_headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",    "Connection": "keep-alive",    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",    "Accept-Language": "zh-CN,zh;q=0.8"    }


timeout1=30
retrynumber=10

In [3]:
def GetArticleList():
    
    urlcat='http://www.0daytown.com/?cat=31&paged=219'
    
    r = requests.get(urlcat,timeout=timeout1,headers=headers1)
    # r.close()
    # r.text
    # print(r.encoding)

    soup=BeautifulSoup(r.text,'lxml')
    # soup
#     print(soup.prettify())

    if "未找到" in soup.find('title'):
        print(soup.find('title'))
    else:
        print('Found')

    ##########找到链接

    tarticle=soup.find(class_='content').find_all('article')#.find("div").find_all('title')
    # print(tarticle)
    titlelist=[]
    for each in tarticle:
    #     print(each)
        print('\n')
    #     each.find(class_="thumbnail").text
        print(each.find('h2').text)
        print(each.find('h2').find('a')['href'])
    #     titlelist.append(each.find(class_="thumbnail").text)
    #     print(titlelist)
    
    print('\n')
    # columnnumbers=len(titlelist)

In [4]:
'''测试上一步函数'''
# GetArticleList()

'测试上一步函数'

In [5]:
'''从每个网页获取百度链接和密码'''

def GetBaiduLinkPwd(url):
    return 1

In [6]:
#根据http://www.0daytown.com/?p=xxxx获取所有清单
#按页面顺序：1~100,101~200,201~300,.....

# totalpagerange=107109 #191220日0daytown网站的最大页面
# 107110~127999 网页无
# 128000开始~130944 up to 191230
# 137461开始~139959 up to 200127 Status:当前时间：2020.1.27-20:55:53 密码过期，无法使用，将来要重新抓取

def GetListbyPageNumbers(eachBatchNumber, PagesPerBatch):
    
    startpagenumber=(eachBatchNumber-1)*PagesPerBatch+1
    endpagenumber=startpagenumber+PagesPerBatch-1
    
    Columns1=['Category','UploadTime','ArticalName','PageLink','BaiduLink','BaiduPwd']
    
    wholepd=pd.DataFrame(columns=Columns1)
    
    print(wholepd)
    baiduinfolist=[]
    
    iTimeSleepCount=0
    
#     startpagenumber=130001
#     endpagenumber=130944
    pagenumber=startpagenumber
    
    maxtrynumber=0
    
    
    
    while pagenumber<=endpagenumber:
    
    
#     for pagenumber in range(startpagenumber,endpagenumber+1): '''这里用for循环,当网络错误时无法修改循环值,改为while循环'''
    
    
        currentPageNumber=pagenumber
        
        iTimeSleepCount+=1
        
        #每抓100次休息30秒
        if iTimeSleepCount==100:
            time.sleep(30)
            iTimeSleepCount=0
            
        
        #除第一次以后，每抓一次休息2~5秒
        if pagenumber!=startpagenumber:
            time.sleep(0.5)
        
        urlpage='http://www.0daytown.com/?p=' + str(pagenumber)
        
        
        print(urlpage)
#         baiduinfolist+=GetBaiduLinkPwd(urlpage)
        #     urlpage='http://www.0daytown.com/?p=5'
#         urlpage=url
    #     print(urlpage)
        try:
            s = requests.Session()
            s.mount('http://', HTTPAdapter(max_retries=retrynumber))
#             r = s.get('http://www.google.com.hk', timeout=5)
#             try:
            
            r = s.get(urlpage,timeout=timeout1,headers=headers1)
            
#             except requests.exceptions.ConnectTimeout:
#                 print('NETWORK_STATUS = False')
            
#             except requests.exceptions.Timeout:
#                 print('REQUEST_TIMEOUT = TRUE')
            
#             except requests.exceptions.ConnectTimeout as e:
#                 response = e.response 
#                 print(e)
#                 print(response.status_code) # should print a status code, instead response is still None
            
#             r = requests.get(urlpage,timeout=timeout1,headers=headers1)
            

            soup=BeautifulSoup(r.text,'lxml')
        #     print(soup)
    #         print(soup.prettify())
            # tarticle=soup.find('body').find('section').find('div').find('div').find('article') #.find('section')#.find('div')
            # tarticle=soup.find(class_='erphpdown-content-vip')
            # tarticle=tarticle.find(class_='erphpdown-content-vip')

            # tarticle=soup.find(class_='erphpdown-content-vip')

            categorynamelist=soup.find(class_='breadcrumbs').find_all('a')
            categoryname=''
            for each in categorynamelist:
                categoryname+=each.text + "->"
            print(categoryname)

            articlename=soup.find(class_='muted')
            articlename=articlename.text
            print(articlename)

            uploadtime=soup.find('time')
            uploadtime=uploadtime.text.strip()
            uploadtime=uploadtime.split('(')[1]
            uploadtime=uploadtime.split(')')[0]
            print(uploadtime)

            tarticle=soup.find(class_="article-content").find_all('p')

            baidulinklist=[]
            baidupwdlist=[]

            for each in tarticle:
    #             print(each)

#                 try:
                #找出baidupan链接
                #如'<p>Download 百度云<br>
                # 链接: <a href="https://pan.baidu.com/s/1rJq2hyT0z8A3AtWye5SCnQ" rel="nofollow" class="external" target="_blank" data-original-title="" title="">https://pan.baidu.com/s/1rJq2hyT0z8A3AtWye5SCnQ</a> 提取码: ezqd</p>'
#                     if 'pan.baidu.com' in each.find('a').text:
#     #                     print(each.find('a').text)
#                         baidulinklist.append(each.find('a').text)

                ahreflist=each.find_all('a')
                for each_a in ahreflist:
                    if 'pan.baidu.com' in each_a.text:
#                     print(each.find('a').text)
                        baidulinklist.append(each_a.text)

                #找出baidupan密码  如 ' 提取码: ituk</p> 
                if '提取码:' in each.text: 
#                     print(each.text)
                    tempstr1=each.text.split('提取码:')[1].strip()
                    baidupwd=tempstr1.split('</p>')[0].strip()
#                     print(baidupwd)
                    baidupwdlist.append(baidupwd)
                elif  '密码:' in each.text: 
                    tempstr1=each.text.split('密码:')[1].strip()
                    baidupwd=tempstr1.split('</p>')[0].strip()
#                     print(baidupwd)
                    baidupwdlist.append(baidupwd)


#                 except:
    #                 print('no pan.baidu')
                    #空指令
#                     a=1

    #             if '链接:' in each.text: 
    #                 string1=each.text
    #                 stringlist1=string1.split('链接: ')[1]
    #                 if  '提取码:' in  stringlist1:
    #                     stringlist2=stringlist1.split(' 提取码: ')
    #                 elif '密码' in stringlist1:
    #                     stringlist2=stringlist1.split(' 密码: ')

    #                 baidulinklist.append(stringlist2[0])
    #                 baidupwdlist.append(stringlist2[1])

    #             elif '百度云'in each.text:
    #                 print each.text
    #                 stringlist1=each.text.split('链接: ')[1]
    #                 if  '提取码:' in  stringlist1:
    #                     stringlist2=stringlist1.split(' 提取码: ')
    #                 elif '密码' in stringlist1:
    #                     stringlist2=stringlist1.split(' 密码: ')

    #                 baidulinklist.append(each.text)
    #                 baidupwdlist.append(stringlist2[1])


            
#             print()

            #新加一行
    #         insertRow = pd.DataFrame([[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist]],columns =Columns1)
    #         insertRow = pd.DataFrame([[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist],columns =Columns1)

            #将List转为字符串
            baidulinkstr=';'.join(baidulinklist)
            baidupwdstr=';'.join(baidupwdlist)

            print(baidulinkstr + '   :    ' + baidupwdstr)
                  
            insertRow = pd.DataFrame([[categoryname,uploadtime,articlename,urlpage,baidulinkstr,baidupwdstr]],columns =Columns1)
    #         print(insertRow)
    #         a=[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist]
            wholepd=wholepd.append(insertRow,ignore_index=True)
            
            pagenumber+=1
        
        except requests.exceptions.ConnectTimeout:
            print('NETWORK_STATUS = False, please wait 5 seconds')
            time.sleep(10)
            pagenumber=currentPageNumber

        except requests.exceptions.Timeout:
            print('REQUEST_TIMEOUT = TRUE, please wait 8 seconds')
            time.sleep(10)
            pagenumber=currentPageNumber

        except Exception as ex:
            print("#############  Exceptions:" + str(ex))
            print('#############  Unknow error in Page: {}, please wait 10 seconds and retry'.format(str(pagenumber)))
        
#         except:
#             print('#############Unknow error in Page: {}, please wait 10 seconds and retry'.format(str(pagenumber)))
            time.sleep(10)
#             print('pagenumber is:' + pagenumber)
#解决如果重新登录10次就跳出该页面
#             print(pagenumber)
#             if pagenumber == 83230 or pagenumber == 84676:
#                 print('#83230这个页面无法绕过 ，一直报错。The.First.Purge.2018.720p.BluRay.x264-DRONES 人类清除计划4 ')
#                 pagenumber+=1
#                 continue
#                 #
#             else:
                
            maxtrynumber+=1

            if maxtrynumber>10:
                print('*******************尝试超过10次无法连接，放弃该链接, PageNumber:' + str(pagenumber))
                maxtrynumber=0
                pagenumber+=1
            else:
                pagenumber=currentPageNumber
        
        
        time_stamp = datetime.datetime.now()
        print("当前时间：" + time_stamp.strftime('%Y.%m.%d-%H:%M:%S'))
            
#         print(wholepd)
#         wholepd.append(insertRow,ignore_index=True)
        
#         wholepd['Category']=categoryname
#         wholepd['ArticalName']=articlename.text
#         wholepd['PageLink']=urlpage
#         baiduinfolist=[baidulinklist,baidupwdlist]
#         return baiduinfolist
#     downloadfilepathname='c:/0daydown_'+ str(startpagenumber)+  '_' + str(endpagenumber) + '.csv"
    downloadfilepathname='c:/0daydown_{0}_{1}.csv'.format(startpagenumber,endpagenumber)
   
    wholepd.to_csv(downloadfilepathname,encoding='UTF-8-sig')
    return 1

In [12]:

'''
批处理X次
Status:当前时间：2019.12.25-20:55:53
Status:当前时间：2020.1.27-20:55:53 密码过期，无法使用，将来要重新抓取
'''
startBatchNumber=13747
BatchNumbers=300
endBatchNumber=startBatchNumber+BatchNumbers

'''每次处理1000个网页，保存为一个文件'''
PagesPerBatch=10

'''按顺序：1~100,101~200,201~300,.....'''
for eachBatchNumber in range(startBatchNumber,endBatchNumber):
    GetListbyPageNumbers(eachBatchNumber, PagesPerBatch)
    
    #每批次休息半分钟
    time.sleep(30)

print('done')

Empty DataFrame
Columns: [Category, UploadTime, ArticalName, PageLink, BaiduLink, BaiduPwd]
Index: []
http://www.0daytown.com/?p=137461
0daytown->教程->
Reymer – Rebel Heart (2019) FLAC
12-28
https://pan.baidu.com/s/150A-Ec1n5aY41cgTddXCAg   :    c8ue
当前时间：2020.01.28-08:32:41
http://www.0daytown.com/?p=137462
0daytown->教程->
The Colorful Quiet – Sounds of the Countryside (2019) FLAC
12-28
https://pan.baidu.com/s/1F-f-IKjpaViIcr0Y10Xhpg   :    iruu
当前时间：2020.01.28-08:32:43
http://www.0daytown.com/?p=137463
0daytown->教程->
Rousse Philharmonic Orchestra feat. Nayden Todorov – Vladigerov: Orchestral Works (2019) FLAC
12-28
https://pan.baidu.com/s/1vlljdwlLfCdQXazcx9c3VQ   :    j1xe
当前时间：2020.01.28-08:32:44
http://www.0daytown.com/?p=137464
0daytown->教程->
Kostas Tosidis – Cycling Modes (2019) FLAC
12-28
https://pan.baidu.com/s/1md59JQ8dGjbnhpBBh8-AiQ   :    fm7i
当前时间：2020.01.28-08:32:46
http://www.0daytown.com/?p=137465
0daytown->教程->
Doug Carn – Infant Eyes (Remastered) (2019) FLAC
12-28
https

KeyboardInterrupt: 

In [ ]:
# Columns1=['Category','ArticalName','PageLink','BaiduLink','BaiduPwd']

# # data={'Category':[1],'ArticalName':[2],'PageLink':[3],'BaiduLink':[4],'BaiduPwd':[5]} 
# wholepd=pd.DataFrame(columns=Columns1)
# # wholepd=  
# print(wholepd)
# print("\n")

# insertRow = pd.DataFrame([[categoryname,articlename.text,urlpage,baidulinklist,baidupwdlist]],columns =Columns1)
# print(insertRow)
# print("\n")
# # a=['categoryname','articlename.text','urlpage','baidulinklist','baidupwdlist']
# # a=[1,2,3,4,5]
# data1={'Category':[3],'ArticalName':[5],'PageLink':[7],'BaiduLink':[8],'BaiduPwd':[9]}

# wholepd=wholepd.append(insertRow,ignore_index=True)
# # wholepd.append([['categoryname','articlename.text','urlpage','baidulinklist','baidupwdlist']],ignore_index=True)

# print(wholepd)